In [4]:
#### FIRST ATTEMPT TO DO THE GEOJSON

import geopandas as gpd

# Check the georeference stystem
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Chile/LasTablas_Fire/Progressoes_LasTablas_v3_COM_GOES.shp')
print(gdf.crs)

# gdf = gdf.to_crs(epsg=4326)
# print(gdf.crs)

# If the result is EPSG:32719 -> Coordenate System iks WGS84 / UTM zone 19S -> NO REPROJECTION NEEDED!!!

EPSG:32719


In [10]:
import geopandas as gpd
import pandas as pd


gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Chile/ColonelFires/Progression_ColonelFire_UTC.shp')

# Rename columns
gdf = gdf.rename(columns={'date_hour': 'iso_timestamp'})
gdf = gdf.drop(['source', 'fire', 'type'], axis=1)

# Add area and perimeter
gdf['area_ha'] = gdf.geometry.area / 10000
gdf['perimeter_m'] = gdf.geometry.length

# Sort by time
gdf = gdf.sort_values('iso_timestamp')

# Add order field
gdf['order_field'] = gdf['iso_timestamp'].rank(ascending=True).astype(int)

# Aggregate dict for dissolve
agg_dict = {}
for col in gdf.columns:
    if col in ['geometry', 'iso_timestamp']:
        continue
    if pd.api.types.is_numeric_dtype(gdf[col].dtype):
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'first'

# Dissolve once by iso_timestamp using agg_dict
gdf_2 = gdf.dissolve(by='iso_timestamp', aggfunc=agg_dict).reset_index()

# Clean geometries
gdf_2['geometry'] = gdf_2.geometry.buffer(0)

# Sort to ensure temporal order
gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)

# Progressive union
geom_union = None
acum_rows = []

for idx, row in gdf_2.iterrows():
    if geom_union is None:
        geom_union = row.geometry
    else:
        geom_union = geom_union.union(row.geometry)
    area_acum = geom_union.area / 10000
    perim_acum = geom_union.length
    acum_rows.append({
        'id': idx + 1,
        'iso_timestamp': row['iso_timestamp'],
        'geometry': geom_union,
        'area_ha': area_acum,
        'perimeter_m': perim_acum
    })

gdf_acum = gpd.GeoDataFrame(acum_rows, crs=gdf_2.crs)

# Convert to WGS84 for export
gdf_acum = gdf_acum.to_crs(epsg=4326)

# Save
gdf_acum.to_file('chile_202302_colonel_acumulado2.geojson', driver='GeoJSON')


In [2]:
# EDIT SHAPEFILE WITH ISOCHRONES # Las Maquinas - Chile
import geopandas as gpd
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Chile/Las_Maquinas/Progression_Las_Maquinas_UTM19S.shp')

# Rename attributes
gdf = gdf.rename(columns={
    'date_hour': 'iso_timestamp'
})

# Delete attribute
gdf = gdf.drop('area', axis=1)
gdf = gdf.drop('source', axis=1)
gdf = gdf.drop('distance', axis=1)
gdf = gdf.drop('duration', axis=1)
gdf = gdf.drop('ROS', axis=1)
gdf = gdf.drop('gr_rate', axis=1)
gdf = gdf.drop('type', axis=1)
gdf = gdf.drop('Area_QGIS', axis=1)

## Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Sort ids with time stamps -> from early to later
gdf = gdf.sort_values('iso_timestamp')

# Prepare agg dict: sumar numéricos, 'first' para el resto
agg_dict = {}
for col in gdf.columns:
    if col in ['geometry', 'iso_timestamp']:
        continue
    if pd.api.types.is_numeric_dtype(gdf[col].dtype):
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'first'

# Convine geometries con la misma 'iso_timestamp'
gdf_2 = gdf.dissolve(by='iso_timestamp', aggfunc=agg_dict)

# devolver iso_timestamp a columna (dissolve lo pone como índice)
gdf_2 = gdf_2.reset_index()

# Recalcular área y perímetro sobre las geometrías combinadas (área de la UNIÓN)
gdf_2['area_ha'] = gdf_2.geometry.area / 10000
gdf_2['perimeter_m'] = gdf_2.geometry.length

# CReate a new 'id' secuancial (starting in 1)
gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

# Define the attribute colums order
mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

# Add the rest of the attributes
extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib]

# Reorganice columns
gdf_2 = gdf_2[mand_atrib + extra_atrib]

# Save GeoJSON
gdf_2 = gdf_2.to_crs(epsg=4326)
gdf_2.to_file('chile_201701_lasmaquinas.geojson', driver='GeoJSON')



In [4]:
# EDIT SHAPEFILE WITH ISOCHRONES # Las Tablas- Chile
import geopandas as gpd
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Chile/LasTablas_Fire/Progressoes_LasTablas_v3_COM_GOES.shp')

# Rename attributes
gdf = gdf.rename(columns={
    'utc_date': 'iso_timestamp'
})

# Delete attribute
gdf = gdf.drop('area_ha', axis=1)
gdf = gdf.drop('source', axis=1)
gdf = gdf.drop('distance', axis=1)
gdf = gdf.drop('duration', axis=1)
gdf = gdf.drop('ros_mh', axis=1)
gdf = gdf.drop('type', axis=1)
gdf = gdf.drop('grate', axis=1)
gdf = gdf.drop('Area_QGIS', axis=1)

## Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Sort ids with time stamps -> from early to later
gdf = gdf.sort_values('iso_timestamp')

# Prepare agg dict: sumar numéricos, 'first' para el resto
agg_dict = {}
for col in gdf.columns:
    if col in ['geometry', 'iso_timestamp']:
        continue
    if pd.api.types.is_numeric_dtype(gdf[col].dtype):
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'first'

# Convine geometries con la misma 'iso_timestamp'
gdf_2 = gdf.dissolve(by='iso_timestamp', aggfunc=agg_dict)

# devolver iso_timestamp a columna (dissolve lo pone como índice)
gdf_2 = gdf_2.reset_index()

# Recalcular área y perímetro sobre las geometrías combinadas (área de la UNIÓN)
gdf_2['area_ha'] = gdf_2.geometry.area / 10000
gdf_2['perimeter_m'] = gdf_2.geometry.length

# CReate a new 'id' secuancial (starting in 1)
gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

# Define the attribute colums order
mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

# Add the rest of the attributes
extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib]

# Reorganice columns
gdf_2 = gdf_2[mand_atrib + extra_atrib]

# Save GeoJSON
gdf_2 = gdf_2.to_crs(epsg=4326)
gdf_2.to_file('chile_202402_lastablas.geojson', driver='GeoJSON')